# Домашнее задание по теме: "Основы веб-скрапинга"

## Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса.

Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [ ]:
# импорт библиотек
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
from IPython.display import display, HTML # для отображения полных строк и кликабельных ссылок
from random import random
import requests
import re

In [ ]:
def get_posts_df(keywords):
    '''Функция получающая данные из поиска на сайте habr.ru'''
    # ссылка
    url = 'https://habr.com'

    # Проверка длины списка
    if len(keywords) == 0:
        print('Список искомых слов пустой!')
        return
    else:
        print(f'Длина полученного списка ключевых слов: {len(keywords)}.')

    # создание экземпляра класса dataframe
    df = pd.DataFrame()

    while len(keywords):
        # fifo
        params = {'q': str(keywords.pop(0))}

        res = requests.get(url + f'/ru/search', params=params)
        soup = BeautifulSoup(res.text)

        # полученние объектов с классом tm-articles-list__item
        articles = soup.find_all('article', class_='tm-articles-list__item') 

        # проход в цикле по объектам articles, получение и запись нужных данных в dataframe
        for article in articles:
            date_full = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
            date = datetime.strptime(str(date_full), '%Y-%m-%dT%H:%M:%S.000Z') # изменение формата даты
            title = article.find('a', class_="tm-article-snippet__title-link").text
            link = url + article.find('a', class_="tm-article-snippet__title-link").get('href')
        
            # итоговый словарь для формирования строки
            row = {'date' : date, 'title' : title, 'link' : link}

            # построчное добавление данных в dataframe
            df = pd.concat([df, pd.DataFrame([row])])

    print(f'Итоговое количество строк в возвращаемом dataframe: {len(df)}.')
    return df

In [ ]:
# список ключевых слов
keywords = ['python', 'js']

habr_posts = get_posts_df(keywords)

Длина полученного списка ключевых слов: 2.
Итоговое количество строк в возвращаемом dataframe: 40.


In [ ]:
# проверка содержания
display(HTML(habr_posts.sample(10).reset_index(drop=True).to_html(render_links=True, escape=False)))

,date,title,link
0,2022-01-20 15:37:16,Курс «Python для инженеров». Старт 3 потока 31 января,https://habr.com/ru/company/southbridge/news/t/646825/
1,2020-12-02 09:29:19,Fwdays'20: Node.js Middleware – никогда больше,https://habr.com/ru/news/t/530930/
2,2020-12-04 18:03:25,Python как компилируемый статически типизированный язык программирования,https://habr.com/ru/news/t/531402/
3,2020-03-03 10:22:32,В начале этого года Python сместил Java и стал вторым по популярности языком программирования среди разработчиков,https://habr.com/ru/company/itsumma/news/t/490834/
4,2016-10-18 10:54:51,Linux Foundation запустил «перезагрузку» JavaScript-сообщества,https://habr.com/ru/news/t/313000/
5,2020-04-21 15:35:14,"Вышел Python 2.7.18, последний релиз ветки Python 2.x",https://habr.com/ru/news/t/498364/
6,2022-01-13 15:35:30,"Открытый урок «Пишем Custom Prometheus Exporter на Python», 19 января",https://habr.com/ru/company/southbridge/news/t/645485/
7,2020-06-03 06:00:30,"Вышла версия 0.0.2 snakeware — дистрибутива Linux, в котором всё работает через Python",https://habr.com/ru/news/t/505096/
8,2022-08-30 10:13:02,Как и почему перешли с Python на Go в основном сервисе рекомендаций Авито,https://habr.com/ru/company/avito/blog/679560/
9,2020-06-22 09:51:17,Клиент Discord модифицировали для кражи аккаунтов,https://habr.com/ru/news/t/507666/


## Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
def get_posts_df(keywords, pages):
    '''Функция получающая данные из поиска на сайте habr.ru'''
    # ссылка
    url = 'https://habr.com'

    # Проверка длины списка
    if len(keywords) == 0:
        print('Список искомых слов пустой!')
        return
    else:
        print(f'Длина полученного списка ключевых слов: {len(keywords)}.')

    # Проверка количества страниц
    if pages <= 0:
        print('Количество страниц: 0 или отрицательное число!')
        return
    else:
        print(f'Количество страниц: {pages}.')

    # создание экземпляра класса dataframe
    df = pd.DataFrame()

    while len(keywords):
        # fifo
        params = {'q': str(keywords.pop(0))}

        for page in range(0, pages):
            res = requests.get(url + f'/ru/search/page{page + 1}', params=params)
            soup = BeautifulSoup(res.text)

            # полученние объектов с классом tm-articles-list__item
            articles = soup.find_all('article', class_='tm-articles-list__item')

            # проход в цикле по объектам articles, получение и запись нужных данных в dataframe
            for article in articles:
                date_full = article.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime')
                date = datetime.strptime(str(date_full), '%Y-%m-%dT%H:%M:%S.000Z') # изменение формата даты
                title = article.find('a', class_="tm-article-snippet__title-link").text
                link = url + article.find('a', class_="tm-article-snippet__title-link").get('href')

                # получение текста материала
                # из-за такой https://habr.com/ru/company/epam_systems/news/t/589555/ ссылки в поиске добавил try/except
                try:
                    # классы постов
                    megapost_c = 'tm-megapost-presenter__megapost-html'
                    other_c = 'article-formatted-body'

                    text_soup = BeautifulSoup(requests.get(link).text)

                    if text_soup.find('div', class_=megapost_c):
                        text = text_soup.find('div', class_=megapost_c).text
                    else:
                        text = text_soup.find('div', class_=other_c).text
                except:
                    print(f'Нерабочая ссылка или редирект: ({link}).')
                    continue

                # получение лайков и дизлайков
                votes_text = article.find('svg', class_='tm-votes-meter__icon').text
                votes = re.findall(r'\d+', votes_text)
                # если список пустой подстановка нулей
                likes, dislikes = (votes[1], votes[2]) if len(votes) != 0 else (0, 0)

                # итоговый словарь для формирования строки
                row = {'date' : date, 'title' : title, 'link' : link, 'text': text, 'likes': likes, 'dislikes': dislikes}

                # построчное добавление данных в dataframe
                df = pd.concat([df, pd.DataFrame([row])])

        # пауза, что-бы избежать блокировку
        time.sleep(0) if pages == 1 else time.sleep(random())

    print(f'Итоговое количество строк в возвращаемом dataframe: {len(df)}.')
    return df

In [ ]:
# список ключевых слов
keywords = ['python для автоматизации', 'js для фронтенда']

# количество страниц для каждого элемента из списка ключевых слов
pages = 5

habr_posts = get_posts_df(keywords, pages)

Длина полученного списка ключевых слов: 2.
Количество страниц: 5.
Итоговое количество строк в возвращаемом dataframe: 200.


In [ ]:
# проверка содержания
display(HTML(habr_posts.sample(10).reset_index(drop=True).to_html(render_links=True, escape=False, formatters={'text': lambda x: str(x)[:256]})))

,date,title,link,text,likes,dislikes
0,2019-03-31 20:47:20,Дайджест свежих материалов из мира фронтенда за последнюю неделю №358 (25 — 31 марта 2019),https://habr.com/ru/post/446202/,"Предлагаем вашему вниманию подборку с ссылками на новые материалы из области фронтенда и около него. Медиа | Веб-разработка | CSS | Javascript | Занимательное Медиа • Подкаст «Веб-стандарты», Выпуск №167: Редакторы, фреймворки",23,0
1,2022-03-24 08:10:43,"Путешествие по камням, или Как мы скрестили криптошлюзы S‑Terra с Ansible для автоматизации конфигурирования устройств",https://habr.com/ru/company/solarsecurity/blog/655753/,"Однажды моей команде довелось организовывать несложную кустовую схему шифрования для компании, у которой было более 2,5 тысяч офисов продаж и около ста региональных центров. Всё техническое описание решения легко излагалось в таблице Excel размером 2 800 с",6,0
2,2016-04-05 10:17:04,Разбираемся в DevOps и Js на примере Dillinger.io,https://habr.com/ru/post/280968/,"Я бэкенд Java-разработчик, и до фронтенда руки доходят очень редко. Это большое упущение. Отсутствие представления о фронтенде не даёт мне увидеть полную картину мира. Я не знаю, на что обращают внимание и как пользуются всеми модными средствами вроде Node",12,0
3,2016-11-06 21:26:07,Дайджест свежих материалов из мира фронтенда за последнюю неделю №235 (1 — 6 ноября 2016),https://habr.com/ru/company/zfort/blog/314552/,"Предлагаем вашему вниманию подборку с ссылками на полезные ресурсы и интересные материалы из области фронтенда. Веб-разработка CSS Javascript Браузеры Новости и зЗанимательное Веб-разработка Подкаст Веб-стандарты, Выпуск №40: Chromium 54,",35,5
4,2017-06-04 20:56:13,Дайджест свежих материалов из мира фронтенда за последнюю неделю №265 (29 мая — 4 июня 2017),https://habr.com/ru/company/zfort/blog/330190/,"Предлагаем вашему вниманию подборку с ссылками на новые материалы из области фронтенда и около него. Веб-разработка CSS Javascript Браузеры Занимательное Веб-разработка Обзор изменений в новом мажорном релизе Node 8 Подкаст Веб-стандарты,",24,2
5,2017-07-31 09:45:32,Почему Node.js в качестве основы фронтенда – это круто [обновлено],https://habr.com/ru/company/yoomoney/blog/334500/,"Накануне запуска школы Node.js от Яндекс.Денег я хотел бы рассказать чуть больше о том, почему именно эта платформа прижилась в нашем фронтенде. Несколько лет назад в Яндекс.Деньгах назрела смена платформы для серверной прослойки фронтенда: имевшаяся была",31,4
6,2019-11-16 06:41:22,Опрос по инструментам фронтенда 2019 — результаты,https://habr.com/ru/post/476048/,TL;DR. В большинстве категорий теперь выделяются явные лидеры — несколько лет назад такого не было. Это помогает накоплению знаний. Поэтому Навыки владения инструментами в среднем становятся глубже у разработчиков всех уровней. В этом году 3005 разработ,37,0
7,2019-03-18 11:29:14,Знакомство с Тестированием в Python. Ч. 3,https://habr.com/ru/company/otus/blog/444204/,"Друзья, у нас для вас отличные новости. Во-первых на улице наконец-то светит солнышко, а это значит, что весна начинает полноправно вступать в свои права. Вторая новость более профильная — уже 20 марта стартует первое занятие в новом потоке по курсу «Разра",18,2
8,2016-08-14 20:43:50,Дайджест свежих материалов из мира фронтенда за последнюю неделю №223 (8 — 14 августа 2016),https://habr.com/ru/company/zfort/blog/307722/,Предлагаем вашему вниманию подборку с ссылками на полезные ресурсы и интересные материалы из области фронтенда Веб-разработка CSS Javascript Браузеры Новости и занимательное Веб-разработка Распространенные ошибки начинающего HTML-верста,40,2
9,2019-06-14 06:16:55,Проверяем сведения о недостоверности в выписках из ЕГРЮЛ. Склеиваем pdf на python,https://habr.com/ru/post/456060/,В настоящее время весьма актуальной темой остается возможность налогового органа исключить из ЕГРЮЛ общество всего лишь ”выявив” в отношении компании так называемые недостоверные сведения. Как показывает статистика с сентября 2018 года ФНС исключила из ЕГР,14,